We want to count the number of oil tanks in Houston TX, a major global oil storage facility.  

Create a GBDX interface using [gbdxtools](https://github.com/digitalglobe/gbdxtools). You need your credentials to do this; you can find them under your profile on gbdx.geobigdata.io.

In [ ]:
import os
os.environ['GBDX_USERNAME'] = 
os.environ['GBDX_PASSWORD'] = 
os.environ['GBDX_CLIENT_ID'] =  
os.environ['GBDX_CLIENT_SECRET'] = 

import gbdxtools
gbdx = gbdxtools.Interface()

Use the catalog to query for imagery over Houston.

In [41]:
W, S, E, N = (-95.14483630657196, 29.696617936567343, -94.98828113079071, 29.773830057098092)

wkt_string = "POLYGON(({} {}, {} {}, {} {}, {} {}, {} {}))".format(W, S, W, N, E, N, E, S, W, S)

filters = ["sensorPlatformName = 'WORLDVIEW03_VNIR'",
           "cloudCover < 10",
           "imageBands = 'Pan_MS1_MS2'"]

results = gbdx.catalog.search(searchAreaWkt=wkt_string,
                              startDate="2016-01-01T00:00:00.000Z",
                              endDate="2016-11-01T00:00:00.000Z",
                              filters=filters)

# Results is a dictionary with multiple entries. Each entry corresponds to an image over the area of interest.
for result in results:
    properties = result['properties']
    print 'catid: ' + properties['catalogID']
    print 'cloud cover: ' + str(properties['cloudCover'])
    print 'off nadir: ' + str(properties['offNadirAngle'])

catid: 104001001838A000
cloud cover: 0
off nadir: 7.41784572601
catid: 10400100183BD500
cloud cover: 0
off nadir: 21.641210556
catid: 1040010023558B00
cloud cover: 0
off nadir: 11.3405036926
catid: 1040010018412D00
cloud cover: 0
off nadir: 10.6088581085


Pick an image; the lowest the cloud cover and the off nadir angle, the better the quality of the results.
We've selected 104001001838A000 (WV3, 2016).

The following batch GBDX workflow orders the raw image from the DG factory, and then produces an orthorectified panchromatic image via the AOP_Strip_Processor. Once the workflow completes, the image is stored under platform-stories/oil-tanks/image-houston-pan. We've already run this workflow for you so you can skip this step if you want and go straight to the oil tank detection part.

In [ ]:
catid = '104001001838A000'

# create order task
# it the images are not on GBDX, this task will order them from the DG factory
order = gbdx.Task('Auto_Ordering')
order.inputs.cat_id = catid
# for this particular task, we need to set this flag to true
order.impersonation_allowed = True

# panchromatic
aop_pan = gbdx.Task('AOP_Strip_Processor')
aop_pan.inputs.data = order.outputs.s3_location.value
aop_pan.inputs.bands = 'PAN'
aop_pan.inputs.enable_acomp = False
aop_pan.inputs.enable_pansharpen = False
aop_pan.inputs.enable_dra = False
aop_pan.inputs.ortho_epsg='UTM'

# define preprocessing workflow
preprocess_wf = gbdx.Workflow([order, aop_pan])

# set output location 
output_location = 'platform-stories/oil-tanks/image-houston'
preprocess_wf.savedata(aop_pan.outputs.data, output_location + '-pan')

# execute
preprocess_wf.execute()
preprocess_wf.status

We'll now use oil-tank-detector to detect oil tanks on this image.

In [44]:
otd = gbdx.Task('oil-tank-detector')
otd.inputs.image = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/oil-tanks/image-houston-pan'
otd.inputs.bbox = ','.join(map(str, (W, S, E, N)))

wf = gbdx.Workflow([otd])
wf.savedata(otd.outputs.detections, 'platform-stories/trial-runs/oil-tanks')
wf.execute()

u'4721672104198972138'

In [50]:
wf.status

{u'event': u'succeeded', u'state': u'complete'}

Once the workflow is complete, download the results.

In [55]:
from os.path import join
gbdx.s3.download(location='platform-stories/trial-runs/oil-tanks/detections', local_dir='detections')

Put the detections on a map.

In [54]:
# Create slippy map
from ipyleaflet import Map, TileLayer, GeoJSON
import json

m = Map(center=[29.71, -95.03], zoom=13)

# This is the Mapbox TMS URL
mapbox_token = 'pk.eyJ1IjoicGxhdGZvcm1zdG9yaWVzIiwiYSI6ImNpeTZkeWlvOTAwNm0yeHFocHFyaGFleDcifQ.wOsbVsS0NXKrWeX2bQwc-g'
url = 'https://a.tiles.mapbox.com/v4/platformstories.oil-tanks-houston/{z}/{x}/{y}.png?access_token=' + mapbox_token

# add raster layer
m.add_layer(TileLayer(url=url))
  
with open('detections/detections.geojson') as f:
    data = json.load(f)
        
n_oil_tanks = len(data['features'])
print 'There are ' + str(n_oil_tanks) + ' oil tanks in the area of interest'

g = GeoJSON(data=data)

# add vector layer
m.add_layer(g)

m        

There are 1428 oil tanks in the area of interest
